In [1]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf
import keras
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
 
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

In [2]:
import pickle

In [3]:
train_df = pd.read_csv("drive/MyDrive/train.csv")
print("Train shape : ",train_df.shape)

Train shape :  (1306122, 3)


In [4]:
## делим выборку на обучающую и тестовую
train_df, val_df = train_test_split(train_df, test_size=0.08, random_state=2022)

## задаем параметры 
embed_size = 300 # длина мах вектора (300, так как используем библиотеки .300)
max_features = 95000 # как много уникальный слов использовать (рандом)
maxlen = 70 # мах кол-во слов в предложении

## заполняем пропущенные значения
train_X = train_df["question_text"].fillna("_##_").values
val_X = val_df["question_text"].fillna("_##_").values

## маркирование
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)

## дополняем предложения до махлен
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)


## выделяем целевые значения
train_y = train_df['target'].values
val_y = val_df['target'].values

In [5]:
# перемешиваем данные
np.random.seed(2022)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]

In [6]:
pred_reg_val_y = pickle.load( open( "drive/MyDrive/save.p", "rb" ) )
reg_val_y = pickle.load( open( "drive/MyDrive/save_vay.p", "rb" ) )

In [7]:
pred_bal_val_y = pickle.load( open( "drive/MyDrive/bal_save.p", "rb" ) )
bal_val_y = pickle.load( open( "drive/MyDrive/bal_save_vay.p", "rb" ) )

In [8]:
pred_cnn_val_y = pickle.load( open( "drive/MyDrive/cnn_save.p", "rb" ) )
cnn_val_y = pickle.load( open( "drive/MyDrive/cnn_save_vay.p", "rb" ) )

In [9]:
pred_lstm_val_y = pickle.load( open( "drive/MyDrive/lstm_save.p", "rb" ) )
lstm_val_y = pickle.load( open( "drive/MyDrive/lstm_save_vay.p", "rb" ) )

In [10]:
pred_fasttext_val_y = pickle.load( open( "drive/MyDrive/fasttext_save.p", "rb" ) )
fasttext_val_y = pickle.load( open( "drive/MyDrive/fasttext_save_vay.p", "rb" ) )

In [11]:
pred_val_y = pickle.load( open( "drive/MyDrive/pred_save.p", "rb" ) )
p_val_y = pickle.load( open( "drive/MyDrive/pred_save_vay.p", "rb" ) )

In [17]:
from matplotlib import pyplot
pyplot.rcParams['figure.figsize']=[12,12]
pyplot.rcParams['figure.dpi'] = 1000

In [18]:
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve

fpr1, tpr1, treshold1 = roc_curve(reg_val_y, pred_reg_val_y)
plt.plot(fpr1, tpr1,label = "ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ")

fpr6, tpr6, treshold6 = roc_curve(bal_val_y, pred_bal_val_y)
plt.plot(fpr6, tpr6,label = "ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ, ОБУЧЕННАЯ НА СБАЛАНСИРОВАННЫХ ДАННЫХ")


fpr2, tpr2, treshold2 = roc_curve(cnn_val_y, pred_cnn_val_y)
plt.plot(fpr2, tpr2, label = "CNN АРХИТЕКТУРА НА GloVe")

fpr3, tpr3, treshold3 = roc_curve(lstm_val_y, pred_lstm_val_y)
plt.plot(fpr3, tpr3,label = "LSTM АРХИТЕКТУРА НА GloVe")

fpr4, tpr4, treshold4 = roc_curve(fasttext_val_y, pred_fasttext_val_y)
plt.plot(fpr4, tpr4,label = "LSTM АРХИТЕКТУРА НА FastText")

fpr5, tpr5, treshold5 = roc_curve(p_val_y,pred_val_y)
plt.plot(fpr5, tpr5,label = "СУММИРУЮЩАЯ МОДЕЛЬ")

plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Пример ROC-кривой')
plt.legend(loc="lower right")
plt.show()